# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [ ]:
import pandas as pd
import numpy as np
import glob, os
import random
pd.options.mode.chained_assignment = None
idx = pd.IndexSlice

In [ ]:
DATA_FOLDER = '../Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

***

### Assumptions

In order to clean the data, we made the following assumptions : 
- A person, counted in one of the three categories (i.e. confirmed, suspect, probable) for deaths and cases, can't be counted in another category. This reasonnable assumption guarantees that we are not counting people multiple times when computing the means.
- As told during the lab session, we consider the Totals/National field for each country as consistent with respect to the different region/cities reports
- Cumulative description/variable (e.g. Total deaths of confirmed) are assumed to carry all items from previous days that has not been registered. More precisely, let d1 and d2 be two dates such that d1 < d2  and d2 - d1 > 1 (with respect to time). Then d2 contains information about the corresponding description from d1 + 1 to d2. 

***

### Loading the data for each country

In [ ]:
def load_country(country_name):
    return pd.concat(map(pd.read_csv, glob.glob(r'' + DATA_FOLDER + '/ebola/' + country_name + '_data/*.csv')))

In [ ]:
#Load the data into a dataframe for each country
guinea_dataframe = load_country('guinea')
liberia_dataframe = load_country('liberia')
sierraleone_dataframe = load_country('sl')

#Print columns label
print('Guinea:', guinea_dataframe.columns.get_values())
print('Liberia:', liberia_dataframe.columns.get_values())
print('Sierra Leone:', sierraleone_dataframe.columns.get_values())

***

### Selecting the interesting descriptions/variables 

In [ ]:
guinea_descriptions = set(guinea_dataframe.Description.values.tolist())
liberia_descriptions = set(liberia_dataframe.Variable.values.tolist())
sierraleone_descriptions = set(sierraleone_dataframe.variable.values.tolist())

print(len(guinea_descriptions))
print(len(liberia_descriptions))
print(len(sierraleone_descriptions))

In [ ]:
# All possible descriptions for guinea
print(guinea_descriptions)

# From which we decided to keep only the following descriptions
filtered_guinea_descriptions = ['New cases of confirmed', 
                                'New cases of suspects', 
                                'New cases of probables',
                                'Total deaths of confirmed',
                                'Total deaths of suspects',
                                'Total deaths of probables']

# Rename dictionary that will be useful later
guinea_rename_dict = {'New cases of confirmed': 'cases_confirmed',
                      'New cases of suspects': 'cases_suspected',
                      'New cases of probables': 'cases_probable',
                      'Total deaths of confirmed': 'deaths_confirmed',
                      'Total deaths of suspects': 'deaths_suspected',
                      'Total deaths of probables': 'deaths_probable'}

For guinea, the descriptions 'New deaths registered today (probables)' and 'New deaths registered today (suspects)' or not really interesting as there are only available for one day and equal to 0. Moreover the description 'New deaths registered today (confirmed)' is also available during only one day and is a redundancy of the description 'New deaths registered today'. Thus we decided to drop those descriptions.

In [ ]:
# All possible descriptions for liberia
print(liberia_descriptions)

# From which we decided to keep only the following descriptions
filtered_liberia_descriptions = ['New case/s (confirmed)', 
                                 'New Case/s (Suspected)', 
                                 'New Case/s (Probable)',
                                 'Total death/s in confirmed cases',
                                 'Total death/s in suspected cases',
                                 'Total death/s in probable cases']

liberia_rename_dict = {'New case/s (confirmed)': 'cases_confirmed',
                       'New Case/s (Suspected)': 'cases_suspected',
                       'New Case/s (Probable)': 'cases_probable',
                       'Total death/s in confirmed cases': 'deaths_confirmed',
                       'Total death/s in suspected cases': 'deaths_suspected',
                       'Total death/s in probable cases': 'deaths_probable'}

In [ ]:
# All possible descriptions for Sierra Leone
print(sierraleone_descriptions)

# From which we decided to keep only the following descriptions
filtered_sierraleone_descriptions = ['death_confirmed', 
                                     'death_probable', 
                                     'death_suspected',
                                     'new_confirmed', 
                                     'new_probable', 
                                     'new_suspected']

sierraleone_rename_dict = {'death_confirmed': 'deaths_confirmed',
                           'death_suspected': 'deaths_suspected',
                           'death_probable': 'deaths_probable',
                           'new_confirmed': 'cases_confirmed',
                           'new_probable': 'cases_probable',
                           'new_suspected': 'cases_suspected'}

***

### Cleaning the dataframes

#### General cleaning

In [ ]:
# Check for duplicates
print('Guinea has duplicates ?', True in guinea_dataframe.duplicated(subset=['Date', 'Description']).values)
print('Liberia has duplicates ?', True in liberia_dataframe.duplicated(subset=['Date', 'Variable']).values)
print('Sierra Leone has duplicates ?', True in sierraleone_dataframe.duplicated(subset=['date', 'variable']).values)

In [ ]:
# Remove duplicates
liberia_dataframe = liberia_dataframe.drop_duplicates(['Date', 'Variable'])
sierraleone_dataframe = sierraleone_dataframe.drop_duplicates(['date', 'variable'])

In [ ]:
# Keep only interesting columns
guinea_dataframe = guinea_dataframe[['Date', 'Description', 'Totals']]
liberia_dataframe = liberia_dataframe[['Date', 'Variable', 'National']]
sierraleone_dataframe = sierraleone_dataframe[['date', 'variable', 'National']]

# Standardize the date field for each dataframe
guinea_dataframe.Date = pd.to_datetime(guinea_dataframe.Date)
liberia_dataframe.Date = pd.to_datetime(liberia_dataframe.Date)
sierraleone_dataframe.date = pd.to_datetime(sierraleone_dataframe.date)

# Keep only the insteresting variables/descriptions
guinea_dataframe = guinea_dataframe[[des in filtered_guinea_descriptions for des in guinea_dataframe.Description]]
liberia_dataframe = liberia_dataframe[[var in filtered_liberia_descriptions for var in liberia_dataframe.Variable]]
sierraleone_dataframe = sierraleone_dataframe[[var in filtered_sierraleone_descriptions for var in sierraleone_dataframe.variable]]

# Remove rows with missing value
guinea_dataframe = guinea_dataframe.dropna()
liberia_dataframe = liberia_dataframe.dropna()
sierraleone_dataframe = sierraleone_dataframe.dropna()

# Cast all values to int
guinea_dataframe.Totals = guinea_dataframe.Totals.astype(int)
liberia_dataframe.National = liberia_dataframe.National.astype(int)
sierraleone_dataframe.National = sierraleone_dataframe.National.astype(int)

# Rename all reports (i.e. variable/description)
guinea_dataframe.Description = guinea_dataframe.Description.apply(lambda des: guinea_rename_dict[des]) 
liberia_dataframe.Variable = liberia_dataframe.Variable.apply(lambda var: liberia_rename_dict[var])
sierraleone_dataframe.variable = sierraleone_dataframe.variable.apply(lambda var: sierraleone_rename_dict[var])


#### Cleaning Guinea dataframe

After looking at the guinea dataframe, we noticed that the original 'Total deaths...' descriptions were cumulative values. We thus had to get the daily new deaths cases for each day. This is what is done in the cell below.

In [ ]:
def clean_cum_guinea(des_name, df):
    r_df = df.copy()
    guinea_parts_to_clean = r_df[r_df.Description == des_name]
    guinea_parts_to_clean = guinea_parts_to_clean.set_index(['Date', 'Description']).sort_index()
    
    cleaned_guinea_parts = (guinea_parts_to_clean - guinea_parts_to_clean.shift(1))
    
    idx = pd.IndexSlice
    r_df = r_df.set_index(['Date', 'Description']).sort_index()
    r_df.loc[idx[:, [des_name]], :] = cleaned_guinea_parts
    r_df = r_df.dropna().reset_index()
    return r_df
    
guinea_dataframe = clean_cum_guinea('deaths_confirmed', guinea_dataframe)
guinea_dataframe = clean_cum_guinea('deaths_suspected', guinea_dataframe)
guinea_dataframe = clean_cum_guinea('deaths_probable', guinea_dataframe)

Notice from the guinea dataframe below that **some reports yield negative values**. Unless they were giving unexpected average values, we generaly decided to interpret them as corrections on previous reports and thus keep them. 

In [ ]:
guinea_dataframe

#### Cleaning Liberia dataframe

In [ ]:
# Last record for in liberia_new_deaths_... is equal to 0 which is a dirty value as it should be greater than the value in the precedent record
# '...' here means {'confirmed', 'probable', 'suspected'}

liberia_dataframe = liberia_dataframe.set_index(['Date', 'Variable']).sort_index()

index0 = liberia_dataframe.loc[idx[:, 'deaths_confirmed'], :][-1:].index
index1 = liberia_dataframe.loc[idx[:, 'deaths_probable'], :][-1:].index
index2 = liberia_dataframe.loc[idx[:, 'deaths_suspected'], :][-1:].index

liberia_dataframe = liberia_dataframe.drop(index0)
liberia_dataframe = liberia_dataframe.drop(index1)
liberia_dataframe = liberia_dataframe.drop(index2)

liberia_dataframe = liberia_dataframe.reset_index()

We can see, from the liberia datafame below, that some rows at the end are dirty as they switch to a cumulative fashion. 

In [ ]:
liberia_dataframe[-30:]

In [ ]:
# This cell deals with the cumulative values found at the end of the dataframe for certain reports

dirty_rows_liberia = liberia_dataframe[liberia_dataframe.Date > pd.datetime(2014, 12, 2)]
dirty_rows_liberia = dirty_rows_liberia.set_index(['Date', 'Variable']).sort_index()

cleaned_rows_liberia = dirty_rows_liberia - dirty_rows_liberia.shift(3)

liberia_dataframe = liberia_dataframe.set_index(['Date', 'Variable']).sort_index()
liberia_dataframe.loc[pd.datetime(2014, 12, 4):, :] = cleaned_rows_liberia
liberia_dataframe = liberia_dataframe.dropna().reset_index()

In [ ]:
# This cell deals with cumulative records (similar to the guinea cleaning case)

def clean_cum_liberia(des_name, df):
    r_df = df.copy()
    liberia_parts_to_clean = r_df[r_df.Variable == des_name]
    liberia_parts_to_clean = liberia_parts_to_clean.set_index(['Date', 'Variable']).sort_index()
    
    cleaned_liberia_parts = (liberia_parts_to_clean - liberia_parts_to_clean.shift(1))
    
    idx = pd.IndexSlice
    r_df = r_df.set_index(['Date', 'Variable']).sort_index()
    r_df.loc[idx[:, [des_name]], :] = cleaned_liberia_parts
    r_df = r_df.dropna().reset_index()
    return r_df

liberia_dataframe = clean_cum_liberia('deaths_confirmed', liberia_dataframe)
liberia_dataframe = clean_cum_liberia('deaths_suspected', liberia_dataframe)
liberia_dataframe = clean_cum_liberia('deaths_probable', liberia_dataframe)

#### Cleaning Sierra Leone dataframe

In [ ]:
# This cell deals with cumulative values (similar to guinea and liberia cases)

sierraleone_dataframe_copy = sierraleone_dataframe.copy().set_index(['date', 'variable']).unstack()
tmp = sierraleone_dataframe.set_index(['date', 'variable']).unstack()
tmp = tmp.National[['deaths_confirmed', 'deaths_probable', 'deaths_suspected']]
cleaned = (tmp - tmp.shift(1)).fillna(value=0)

sierraleone_dataframe_copy.National.loc[1:, ['deaths_confirmed', 'deaths_probable', 'deaths_suspected']] = cleaned[1:]
sierraleone_dataframe = sierraleone_dataframe_copy.stack().reset_index()[3:]

In [ ]:
# Remove first three lines 
sierraleone_dataframe = sierraleone_dataframe[3:]

***

### Aggregation and merging

In [ ]:
# Keeps only the year and month of each report
guinea_dataframe['Period'] = guinea_dataframe.Date.dt.to_period('M')
liberia_dataframe['Period'] = liberia_dataframe.Date.dt.to_period('M')
sierraleone_dataframe['Period'] = sierraleone_dataframe.date.dt.to_period('M')

# Keep only the day number of the month. It will be necessary when computing the means
guinea_dataframe.Date = guinea_dataframe.Date.apply(lambda x: x.day)
liberia_dataframe.Date = liberia_dataframe.Date.apply(lambda x: x.day)
sierraleone_dataframe.date = sierraleone_dataframe.date.apply(lambda x: x.day)

# Add a country column to each dataframe. This is required to merge the 3 different dataframes.
guinea_dataframe['Country'] = 'Guinea'
liberia_dataframe['Country'] = 'Liberia'
sierraleone_dataframe['Country'] = 'Sierraleone'

In [ ]:
# Compute averages for each month
averages_guinea_dataframe = pd.DataFrame(guinea_dataframe.groupby(['Country', 'Period', 'Description']).apply(lambda group: max(group['Totals'].sum() / group['Date'].max(), 0)))
averages_liberia_dataframe = pd.DataFrame(liberia_dataframe.groupby(['Country', 'Period', 'Variable']).apply(lambda group: group['National'].sum() / group['Date'].max()))
averages_sierraleone_dataframe = pd.DataFrame(sierraleone_dataframe.groupby(['Country', 'Period', 'variable']).apply(lambda group: group['National'].sum() / group['date'].max()))

# Remove index
averages_guinea_dataframe = averages_guinea_dataframe.reset_index()
averages_liberia_dataframe = averages_liberia_dataframe.reset_index()
averages_sierraleone_dataframe = averages_sierraleone_dataframe.reset_index()

# Rename columns
averages_guinea_dataframe.columns = ['country', 'date', 'report', 'average']
averages_liberia_dataframe.columns = ['country', 'date', 'report', 'average']
averages_sierraleone_dataframe.columns = ['country', 'date', 'report', 'average']

# Stack all averages into one dataframe
all_averages_dataframe = pd.concat([averages_guinea_dataframe, averages_liberia_dataframe, averages_sierraleone_dataframe])

# Sort final dataframe, set index and unstack result (for readibility reasons)
all_averages_dataframe = all_averages_dataframe.sort_values(['date']).set_index(['country', 'date', 'report']).unstack()


In [ ]:
all_averages_dataframe

***

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here